In [270]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [271]:
import pandas as pd

df = pd.read_json("mat_data_cc.json")
df.head(3)

,comments,discourses,discussion_id,discussion_name,number_of_comments,topics
0,"[{'comment_id': 0, 'comment': 'A structure or ...","[{'discourse_name': 'question', 'count': 7, 'c...",1,Build a tower,36,"[{'topic_phrase': 'increased mobility', 'topic..."
1,"[{'comment_id': 36, 'comment': 'This could be ...","[{'discourse_name': 'question', 'count': 0, 'c...",2,Public Art,23,"[{'topic_phrase': 'bad art', 'topic_keywords':..."
2,"[{'comment_id': 59, 'comment': 'It's more diff...","[{'discourse_name': 'question', 'count': 1, 'c...",3,Roundabout,26,"[{'topic_phrase': 'roundabouts', 'topic_keywor..."


In [272]:
prop_data = df.loc[df["discussion_name"] == "Roundabout"]
comment_list = prop_data.comments.tolist()

In [273]:
documents = [c["comment"] for c in comment_list[0]]
doc = comment_list[17]

In [274]:
from collections import defaultdict
from gensim import corpora

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

2020-07-18 15:39:41,441 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-07-18 15:39:41,443 : INFO : built Dictionary(98 unique tokens: ['if', 'it', "it's", 'makes', 'more']...) from 26 documents (total 360 corpus positions)


In [275]:
from gensim import models
lsi = models.LsiModel(corpus, id2word=dictionary)

2020-07-18 15:39:41,615 : INFO : using serial LSI version on this node
2020-07-18 15:39:41,616 : INFO : updating model with new documents
2020-07-18 15:39:41,616 : INFO : preparing a new chunk of documents
2020-07-18 15:39:41,617 : INFO : using 100 extra samples and 2 power iterations
2020-07-18 15:39:41,618 : INFO : 1st phase: constructing (98, 300) action matrix
2020-07-18 15:39:41,620 : INFO : orthonormalizing (98, 300) action matrix
2020-07-18 15:39:41,628 : INFO : 2nd phase: running dense svd on (98, 26) matrix
2020-07-18 15:39:41,629 : INFO : computing the final decomposition
2020-07-18 15:39:41,630 : INFO : keeping 26 factors (discarding 0.000% of energy spectrum)
2020-07-18 15:39:41,630 : INFO : processed documents up to #26
2020-07-18 15:39:41,631 : INFO : topic #0(15.032): 0.372*"is" + 0.334*"be" + 0.283*"traffic" + 0.255*"motor" + 0.232*"as" + 0.215*"that" + 0.194*"on" + 0.167*"could" + 0.149*"into" + 0.134*"they"
2020-07-18 15:39:41,633 : INFO : topic #1(7.406): -0.226*"if"

In [276]:
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]  # convert the query to LSI space
print(vec_lsi)

doc

[(0, 0.3897074696319121), (1, 0.05955613948181116), (2, 0.417216427338147), (3, 0.11399014433494459), (4, 0.4736872809308651), (5, -0.360144719753604), (6, 0.4220228833126771), (7, -0.2281254377024097), (8, -0.03645113669221394), (9, 0.4077976609480935), (10, 0.3781213579425288), (11, 0.2412549128507068), (12, 0.30102098051382664), (13, -1.0750106657104086), (14, 0.4467463027776967), (15, -0.04503310672114176), (16, 0.05692860722114666), (17, 0.9041561437674349), (18, 0.950029716545209), (19, -0.6969425800150169), (20, 0.05162062325535405), (21, 0.04254517426025235), (22, 0.20028701716728597), (23, 0.04328242240485548), (24, 0.00724299747899831), (25, -0.03271439955500394)]


'Probably makes it worse due to the larger distances and awkward crossings required at roundabouts.'

In [277]:
from gensim import similarities
index = similarities.MatrixSimilarity(lsi[corpus])  # transform corpus to LSI space and index it

2020-07-18 15:39:41,910 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2020-07-18 15:39:41,912 : INFO : creating matrix with 26 documents and 26 features
C:\Users\ziabd\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [278]:
sims = index[vec_lsi]  # perform a similarity query against the corpus
print(list(enumerate(sims)))  # print (document_number, document_similarity) 2-tuples

[(0, 0.31622776), (1, 0.10259784), (2, -1.8721039e-08), (3, 0.27975145), (4, 0.13187611), (5, -7.21775e-09), (6, -7.0244823e-09), (7, 0.1490712), (8, 0.31622776), (9, 0.16903085), (10, 0.14142135), (11, 0.06388767), (12, 2.64572e-09), (13, 0.44721362), (14, 0.09534626), (15, 0.1700051), (16, 1.3315002e-09), (17, 1.0), (18, 1.4770194e-09), (19, 0.111803375), (20, 1.8117134e-08), (21, 0.18257421), (22, -4.0308805e-09), (23, -2.6775524e-08), (24, 8.1490725e-09), (25, 8.600182e-09)]


In [287]:
#fix this

sim_obj_list = []

sorted_sims = sorted(enumerate(sims), key=lambda item: -item[1])
for i, s in enumerate(sorted_sims):
#     print(s, documents[s[0]])
    sim_obj_list.append({"id": s[0], "comment": documents[s[0]], "score": str(s[1])})

sim_obj_list

[{'id': 17,
  'comment': 'Probably makes it worse due to the larger distances and awkward crossings required at roundabouts.',
  'score': '1.0'},
 {'id': 13, 'comment': "It won't.", 'score': '0.44721362'},
 {'id': 0,
  'comment': "It's more difficult than it seems and only makes sense if there's a lot of traffic.",
  'score': '0.31622776'},
 {'id': 8,
  'comment': 'None.  It enables pedestrians and vehicles to coexist without building bridges.',
  'score': '0.31622776'},
 {'id': 3,
  'comment': 'As long as we can make it work from a traffic perspective, I love it. It would transform the entire area and it is a very important connection point in downtown.',
  'score': '0.27975145'},
 {'id': 21,
  'comment': 'its a traffic circulation element, not a place. not about human interactions at all.',
  'score': '0.18257421'},
 {'id': 15,
  'comment': "I have come up with these idea to maximize safety. Currently downtown is very unsafe due to the rush of motor traffic on every street as well as

['This could be an obelisk or major art piece authored by Barrio artists.',
 'It is envisioned as a small traffic roundabout with a major art piece in the center in the spirit of Tijuana or Mexico City.',
 'Reach out to the School system (i.e. FIDM, NewSchool, Community College, soon-to-be UCSD)',
 "I think it's a great idea. Will provide a space for exchanging ideas and cultural conversations.",
 'Relevant and great.',
 'Public art always enlivens the area and sets a positive tone.  Just think of Millennial Park in Chicago.',
 'AGAIN---COPYING ANOTHER IDEA....I think it should be something NEVER DONE BEFORE',
 'Like this idea, but you will need to \\curate\\" or make sure the art is appropriate."',
 'I think its feasible, there is a lot of local artists',
 'It can bring in tourist so it may impact the families who live there.',
 'By beautifying the neighborhood, and having the community participate in the \\making\\" will increase pride and care for all."',
 'I dont think so.',
 'more